**Sprint 2    MODEL BUILDING**

**FOR BODY DAMAGE**

1.Import The ImageDataGenerator Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive")
Dataset = 'Car damage'
Data_Dir=os.listdir(Dataset)
print(Data_Dir)

['body', 'level']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Car damage/body/training',
                                                 target_size=(224, 224),
                                                 batch_size=10,
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Car damage/body/validation',
                                            target_size=(224, 224),
                                            batch_size=10,
                                            class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


**MODEL BUILDING**

1.Importing The Model Building Libraries

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

2.Loading The Model

In [ ]:
imagesize = [224, 224]

train_path = '/content/drive/MyDrive/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Dataset/body/training'
valid_path = '/content/drive/MyDrive/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Dataset/body/validation'

In [ ]:
#adding preprocessing Layers to the front of vgg

vgg16 =VGG16(input_shape=imagesize + [3],weights='imagenet',include_top= False)

58889256/58889256 [==============================] - 0s 0us/step


3.Adding Flatten Layer

In [ ]:
#don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
folders=glob('/content/drive/MyDrive/Car damage/body/training/*')

In [ ]:
folders

['/content/drive/MyDrive/Car damage/body/training/02-side',
 '/content/drive/MyDrive/Car damage/body/training/00-front',
 '/content/drive/MyDrive/Car damage/body/training/01-rear']

In [ ]:
#Our Layers -you can add more if you want
x = Flatten()(vgg16.output)

In [ ]:
len(folders)

3

4.Adding Output Layer

In [ ]:
prediction =Dense(len(folders),activation ='softmax')(x)

5.Creating A Model Object

In [ ]:
# create a model object
model =Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

6.Configure The Learning Process

In [ ]:
#Compiling the CNN Model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

7.Train the Model

In [ ]:
%%capture
from time import sleep
import random
sleep(0.1)

In [ ]:
import sys
#fit the model
r = model.fit_generator ( 
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 622s 6s/step - loss: 1.2631 - accuracy: 0.5271 - val_loss: 0.8213 - val_accuracy: 0.6842
Epoch 2/25
98/98 [==============================] - 615s 6s/step - loss: 0.7167 - accuracy: 0.7232 - val_loss: 0.9808 - val_accuracy: 0.6374
Epoch 3/25
98/98 [==============================] - 615s 6s/step - loss: 0.5639 - accuracy: 0.7794 - val_loss: 0.8704 - val_accuracy: 0.6667
Epoch 4/25
98/98 [==============================] - 611s 6s/step - loss: 0.3731 - accuracy: 0.8539 - val_loss: 1.2635 - val_accuracy: 0.5789
Epoch 5/25
98/98 [==============================] - 610s 6s/step - loss: 0.3092 - accuracy: 0.8907 - val_loss: 1.2483 - val_accuracy: 0.5965
Epoch 6/25
98/98 [==============================] - 608s 6s/step - loss: 0.3043 - accuracy: 0.8917 - val_loss: 1.0157 - val_accuracy: 0.6667
Epoch 7/25
98/98 [==============================] - 612s 6s/step - loss: 0.2238 - accuracy: 0.9254 - val_loss: 1.0529 - val_accuracy: 0.6667
Epoch 8/25
98

8. Save The Model

In [ ]:
from tensorflow.keras.models import load_model

model.save('body.h5')

9. Test The Model

In [ ]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [ ]:
model = load_model('body.h5')


In [ ]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [ ]:
import numpy as np

In [ ]:
data = "/content/drive/MyDrive/Car damage/body/training/00-front/0010.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 1s/step
front


**FOR LEVEL DAMAGE**

1. Import The ImageDataGenerator Library


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2. Configure ImageDataGenerator Class

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

3. Apply ImageDataGenerator Functionality To Trainset And Testset

In [5]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Car damage/level/training',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Car damage/level/validation',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


MODEL BUILDING
1. Importing The Model Building Libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

2. Loading The Model


In [6]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Car_damage/level/training'
valid_path = '/content/drive/MyDrive/Car_damage/level/validation'

In [7]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


3. Adding Flatten Layer

In [8]:
for layer in vgg16.layers:
    layer.trainable = False

In [9]:
folders = glob('/content/drive/MyDrive/Car damage/level/training/*')

In [10]:
folders

['/content/drive/MyDrive/Car damage/level/training/02-moderate',
 '/content/drive/MyDrive/Car damage/level/training/03-severe',
 '/content/drive/MyDrive/Car damage/level/training/01-minor']

In [11]:
x = Flatten()(vgg16.output)

In [12]:
len(folders)

3

4. Adding Output Layer

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

5. Creating A Model Object

In [14]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

6. Configure The Learning Process

In [16]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

8. Save The Model

In [19]:
from tensorflow.keras.models import load_model

model.save('level.h5')

9. Test The Model

In [20]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [21]:
model = load_model('level.h5')

In [22]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["minor","moderate","severe"]
  preds = label[np.argmax(prediction)]
  return preds

In [23]:
import numpy as np

In [25]:
data = "/content/drive/MyDrive/Car damage/level/validation/01-minor/0002.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 797ms/step
moderate
